In [234]:
import numpy as np
import pandas as pd
from itertools import product
#from nltk import ngrams
#import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [235]:
#for 95% probability
t = 2

# Calculate n grams

In [236]:
alphabet = 'абвгдеёжзийклмнопрстуфхцчьыъшщэюя'
n = 2

def build_n_grams(n):
    combinations = [''.join(val) for val in [p for p in product(list(alphabet),repeat = n)]]
    return combinations

n_grams = build_n_grams(n)

# Read textes and build dataframe

In [237]:
def build_dataframe(text: str, gramms):
    text_data = {}
    lenght = len(text)
    for i in gramms:
        text_data[i] = text.count(i)/lenght
    return text_data

In [238]:
def read_text(filepath, n_grams):
    data = {}
    with open(filepath, 'r') as f:
        try:
            text = ''.join(filter(str.isalpha, f.read()))
            data['lenght'] = len(text)
            data.update(build_dataframe(text, n_grams))
        except UnicodeDecodeError as e: 
            print('Unable to read the text (some encoding errors)')
            print('\033[91m {} \033[0m'.format(str(e)))
    return data

In [239]:
def read_authors(dirname, n_grams):
    data = []
    for _dir in os.listdir(dirname):
        _current_dir = os.path.join(dirname,_dir)
        if(os.path.isdir(_current_dir)):
            for _file in os.listdir(_current_dir):
                if '.DS_Store' in _file:
                    print('\033[91m {} \033[0m'.format(str('.DS_Store file')))
                else:
                    filepath = os.path.join(_current_dir, _file)
                    text = None
                    string_data = {'author': _dir, 'name': _file}
                    string_data.update(read_text(filepath, n_grams))
                    data.append(string_data)
    return data

# Read data for prediction 

In [240]:
def read_unknown(path, n_grams):
    data = []
    for _file in os.listdir(path):
        if '.DS_Store' in _file:
            print('\033[91m {} \033[0m'.format(str('.DS_Store file')))
        else:
            row = {'name': _file}
            row.update(read_text(os.path.join(path, _file), n_grams))
            data.append(row)
    return data

# Build model

In [241]:
data = read_authors('./50 000', n_grams)
df = pd.DataFrame(data)
unique_authors = df.author.unique()

 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 
 .DS_Store file 


In [242]:
unique_authors

array(['Хаскли', 'Ильин', 'Калинина', 'Курпатов', 'Агапкин', 'Марк Фрост',
       'Достоевский', 'Тургенев', 'Генри Миллер', 'Александрова',
       'Казанцев'], dtype=object)

In [243]:
df.head()

,author,lenght,name,аа,аб,ав,аг,ад,ае,аж,...,ёч,ёш,ёщ,ёъ,ёы,ёь,ёэ,ёю,ёя,ёё
0,Хаскли,42479,Хаксли О. Слепец в Газе.,0.000235,0.001530,0.004802,0.001342,0.002919,0.001695,0.001577,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Хаскли,44158,Хаксли О. Гений и богиня. часть 3,0.000453,0.001540,0.004235,0.001834,0.002650,0.001993,0.001449,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Хаскли,44298,Хаксли О. Гений и богиня. часть 2,0.000361,0.001625,0.004808,0.001467,0.003070,0.001919,0.001558,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Хаскли,41381,Хаксли О. О дивный новый мир. часть 2,0.000217,0.001643,0.005147,0.001184,0.003262,0.001474,0.001643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Хаскли,41684,Хаксли О. О дивный новый мир. часть 3,0.000192,0.002015,0.004150,0.001367,0.003814,0.001056,0.002183,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
df_mean = df.groupby('author').mean()
df_mean

,lenght,ааа,ааб,аав,ааг,аад,аае,ааж,ааз,ааи,...,ёёч,ёёш,ёёщ,ёёъ,ёёы,ёёь,ёёэ,ёёю,ёёя,ёёё
author,,,,,,,,,,,,,,,,,,,,,
Гоголь,15289.0,0.0,0.0,0.000037,0.0,0.0,0.000000,0.0,0.000012,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Достоевский,13343.2,0.0,0.0,0.000022,0.0,0.0,0.000011,0.0,0.000011,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Толстой,8822.6,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000025,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
df_mean.values[:,1:]

array([[0.00000000e+00, 0.00000000e+00, 3.66468163e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.20191567e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [189]:
df_mean.iterrows()

<generator object DataFrame.iterrows at 0x13ecb9480>

In [219]:
df_predict

,lenght,name,ааа,ааб,аав,ааг,аад,аае,ааж,ааз,...,ёёч,ёёш,ёёщ,ёёъ,ёёы,ёёь,ёёэ,ёёю,ёёя,ёёё
0,37392,Советские помещики - Гоголь,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,63233,Слабое сердце - Достоевский,0.000016,0.0,0.000000,0.0,0.000016,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,36907,Петербургские сновидения - Достоевский,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000027,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21759,Ползунков - Достоевский,0.000000,0.0,0.000046,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20099,Повесть о том - Гоголь,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [181]:
pr = df_predict.drop(['lenght', 'name'], axis=1)

In [215]:
for line in pr.keys():
    print()
    for author, val in df_mean.iterrows():
        array = np.abs(line-np.array(val)[1:]) - authors_interval[author]
        print(author, len(np.where(array >= 0)[0])/len(array))
        

Гоголь 0.8019311573030581
Достоевский 0.8132287057906893
Толстой 0.8349333555945126
Гоголь 0.8029329103709268
Достоевский 0.812199126248713
Толстой 0.8354620586025545
Гоголь 0.8036285722236135
Достоевский 0.8119765144558533
Толстой 0.8354898850766619
Гоголь 0.8046581517655897
Достоевский 0.8145365500737402
Толстой 0.8367699028856054
Гоголь 0.803712051645936
Достоевский 0.8148982942371372
Толстой 0.8356290174471993


In [121]:
authors_data = {value: [] for value in unique_authors}
authors_count = {value: 0 for value in unique_authors}
authors_interval = {value: [] for value in unique_authors}
for index, row in df.iterrows():
    row_author = pow(row-df_mean.stack()[row['author']],2)
    del(row_author['author'])
    del(row_author['lenght'])
    authors_count[row['author']] += 1
    authors_data[row['author']].append(row_author)
for i in authors_data:
    authors_interval[i] = 2*np.sum(pow(np.array(authors_data[i])/authors_count[i], 0.5), axis = 0)

In [136]:
authors_interval

{'Гоголь': array([0.0, 0.0, 0.0002622232717544417, ..., 0.0, 0.0, 0.0], dtype=object),
 'Толстой': array([0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0], dtype=object),
 'Достоевский': array([0.0, 0.0, 0.00015755625956180398, ..., 0.0, 0.0, 0.0], dtype=object)}

In [141]:
for index in df_mean.values:
#     auth = ?
    print(index)

[15289.     0.     0. ...     0.     0.     0.]
[13343.2     0.      0.  ...     0.      0.      0. ]
[8822.6    0.     0.  ...    0.     0.     0. ]


In [132]:
predict['Повесть о том - Гоголь']

[{'lenght': 20099,
  'ааа': 0.0,
  'ааб': 0.0,
  'аав': 0.0,
  'ааг': 0.0,
  'аад': 0.0,
  'аае': 0.0,
  'ааё': 0.0,
  'ааж': 0.0,
  'ааз': 0.0,
  'ааи': 0.0,
  'аай': 0.0,
  'аак': 0.0,
  'аал': 0.0,
  'аам': 0.0,
  'аан': 0.0,
  'аао': 0.0,
  'аап': 0.0,
  'аар': 0.0,
  'аас': 4.975371909050202e-05,
  'аат': 4.975371909050202e-05,
  'аау': 0.0,
  'ааф': 0.0,
  'аах': 0.0,
  'аац': 0.0,
  'аач': 0.0,
  'ааь': 0.0,
  'ааы': 0.0,
  'ааъ': 0.0,
  'ааш': 0.0,
  'аащ': 0.0,
  'ааэ': 0.0,
  'ааю': 0.0,
  'аая': 0.0,
  'аба': 0.0008955669436290362,
  'абб': 0.0,
  'абв': 0.0,
  'абг': 0.0,
  'абд': 0.0,
  'абе': 0.0,
  'абё': 0.0,
  'абж': 0.0,
  'абз': 0.0,
  'аби': 0.0,
  'абй': 0.0,
  'абк': 0.0,
  'абл': 0.0,
  'абм': 0.0,
  'абн': 0.0,
  'або': 0.0003482760336335141,
  'абп': 0.0,
  'абр': 0.0,
  'абс': 0.0,
  'абт': 0.0,
  'абу': 0.00024876859545251007,
  'абф': 0.0,
  'абх': 0.0,
  'абц': 0.0,
  'абч': 0.0,
  'абь': 0.0,
  'абы': 0.00024876859545251007,
  'абъ': 0.0,
  'абш': 0.0,
  '

# Prediction

In [232]:
predict = read_unknown('./predict', n_grams)

 .DS_Store file 


In [233]:
df_predict = pd.DataFrame(predict)
df_predict

,lenght,name,ааа,ааб,аав,ааг,аад,аае,ааж,ааз,...,ёёч,ёёш,ёёщ,ёёъ,ёёы,ёёь,ёёэ,ёёю,ёёя,ёёё
0,40476,Александрова Н. Миллион черных роз,0.000000,0.000000,0.000000,0.0,0.000025,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,63233,Слабое сердце - Достоевский,0.000016,0.000000,0.000000,0.0,0.000016,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,42466,Марк Фрост. Шесть Мессий часть 1 4.20.53 PM,0.000000,0.000024,0.000024,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21759,Ползунков - Достоевский,0.000000,0.000000,0.000046,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
